In [8]:
import requests
import json

from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity


In [4]:
corpus_of_documents = [
    "Take a leisurely walk in the park and enjoy the fresh air.",
    "Visit a local museum and discover something new.",
    "Attend a live music concert and feel the rhythm.",
    "Go for a hike and admire the natural scenery.",
    "Have a picnic with friends and share some laughs.",
    "Explore a new cuisine by dining at an ethnic restaurant.",
    "Take a yoga class and stretch your body and mind.",
    "Join a local sports league and enjoy some friendly competition.",
    "Attend a workshop or lecture on a topic you're interested in.",
    "Visit an amusement park and ride the roller coasters."
]

In [ ]:
user_input = "I don't like to hike"
relevant_document = return_response(user_input, corpus_of_documents)
print(relevant_document)
# https://github.com/jmorganca/ollama/blob/main/docs/api.md
full_response = []
prompt = """
You are a bot that makes recommendations for activities. You answer in very short sentences and do not include extra information.
This is the recommended activity: {relevant_document}
The user input is: {user_input}
Compile a recommendation to the user based on the recommended activity and the user input.
"""
url = 'http://localhost:11434/api/generate'
data = {
    "model": "llama2",
    "prompt": prompt.format(user_input=user_input, relevant_document=relevant_document)
}
print(data)
headers = {'Content-Type': 'application/json'}
response = requests.post(url, data=json.dumps(data), headers=headers, stream=True)
try:
    for line in response.iter_lines():
        # filter out keep-alive new lines
        if line:
            decoded_line = json.loads(line.decode('utf-8'))
            print(decoded_line)
            # print(decoded_line['response'])  # uncomment to results, token by token
            full_response.append(decoded_line['response'])
finally:
    response.close()
print(''.join(full_response))

## Semantic Search

In [5]:
model = SentenceTransformer('all-MiniLM-L6-v2')
doc_embeddings = model.encode(corpus_of_documents)

c:\Users\NailFerroukhi\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\models\bert\modeling_bert.py:435: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


In [6]:
print(doc_embeddings)

[[ 0.07121076 -0.01088004  0.11746486 ...  0.0141492  -0.13175769
  -0.00402592]
 [ 0.04881531 -0.03166635  0.07468717 ... -0.0627827  -0.11120288
   0.03045148]
 [ 0.05019964 -0.09127749  0.08517752 ...  0.01286449 -0.07415231
  -0.06140353]
 ...
 [ 0.05416268 -0.03030901  0.02475947 ... -0.01272298 -0.06512283
   0.05848261]
 [-0.00401902 -0.04562394 -0.00900758 ...  0.03939749 -0.12731634
   0.05255727]
 [ 0.05046039  0.01430449  0.0878795  ... -0.01778715 -0.05246405
  -0.02887326]]


### Cosine similarity

In [9]:
query = "What's the best outside activity?"
query_embedding = model.encode([query])
similarities = cosine_similarity(query_embedding, doc_embeddings)

In [10]:
print(similarities)

[[0.50235206 0.3282639  0.31544408 0.50193346 0.44371963 0.18485206
  0.2104584  0.25540653 0.22164026 0.45777765]]


### Ranking and Recommending Activities

Sorting Similarity Scores

In [11]:
indexed = list(enumerate(similarities[0]))
sorted_index = sorted(indexed, key=lambda x: x[1], reverse=True)

In [12]:
print(sorted_index)

[(0, 0.50235206), (3, 0.50193346), (9, 0.45777765), (4, 0.44371963), (1, 0.3282639), (2, 0.31544408), (7, 0.25540653), (8, 0.22164026), (6, 0.2104584), (5, 0.18485206)]


Selecting Top-Ranked Documents

In [13]:
recommended_documents = []
for value, score in sorted_index:
    formatted_score = "{:.2f}".format(score)
    print(f"{formatted_score} => {corpus_of_documents[value]}")
    if score > 0.3:
        recommended_documents.append(corpus_of_documents[value])

0.50 => Take a leisurely walk in the park and enjoy the fresh air.
0.50 => Go for a hike and admire the natural scenery.
0.46 => Visit an amusement park and ride the roller coasters.
0.44 => Have a picnic with friends and share some laughs.
0.33 => Visit a local museum and discover something new.
0.32 => Attend a live music concert and feel the rhythm.
0.26 => Join a local sports league and enjoy some friendly competition.
0.22 => Attend a workshop or lecture on a topic you're interested in.
0.21 => Take a yoga class and stretch your body and mind.
0.18 => Explore a new cuisine by dining at an ethnic restaurant.


### Integrating with a Language Model for Natural Responses

The Prompt Engineering Process

In [15]:
prompt = """
You are a bot that makes recommendations for activities. You answer in very short sentences and do not include extra information.
These are potential activities:
{recommended_activities}
The user's query is: {user_input}
Provide the user with 2 recommended activities based on their query.
"""
recommended_activities = "\n".join(recommended_documents)

Generating the Final Recommendation

In [16]:
user_input = "I like to hike"
full_prompt = prompt.format(user_input=user_input, recommended_activities=recommended_activities)

Then run model execution:

In [17]:
url = 'http://localhost:11434/api/generate'
data = {
    "model": "llama2",
    "prompt": full_prompt
}
headers = {'Content-Type': 'application/json'}
response = requests.post(url, data=json.dumps(data), headers=headers, stream=True)
full_response=[]
try:
    count = 0
    for line in response.iter_lines():
        #filter out keep-alive new lines
        # count += 1
        # if count % 5== 0:
        #     print(decoded_line['response']) # print every fifth token
        if line:
            decoded_line = json.loads(line.decode('utf-8'))
            
            full_response.append(decoded_line['response'])
finally:
    response.close()
print(''.join(full_response))

 Great! Based on your interest in hiking, here are two recommended activities for you:

1. Go for a hike and admire the natural scenery.
2. Visit an amusement park and ride the roller coasters.


: 